In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import re

# Đọc dữ liệu từ file CSV
df = pd.read_csv('requests1.csv', sep=',')

# Chuyển Timestamp sang datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%Y/%m/%d - %H:%M:%S')

def convert_time_to_seconds(time_str):
    # Loại bỏ các khoảng trắng và đảm bảo chuỗi có dạng chuẩn
    time_str = time_str.strip()

    if 's' in time_str:
        # Nếu có đơn vị giây (s), tách số trước 's'
        if 'm' in time_str:
            # Trường hợp có cả phút (m) và giây (s) như "2m6s"
            minutes = int(time_str.split('m')[0])  # lấy phút
            seconds = float(time_str.split('m')[1].replace('s', ''))  # lấy giây
            return minutes * 60 + seconds
        else:
            # Trường hợp chỉ có giây
            return float(time_str.replace('s', ''))
    elif 'ms' in time_str:
        # Trường hợp mili giây (ms), chuyển thành giây
        return float(time_str.replace('ms', '')) / 1000
    elif 'µs' in time_str:
        # Trường hợp micro giây (µs), chuyển thành giây
        return float(time_str.replace('µs', '')) / 1_000_000
    else:
        # Nếu không có đơn vị, trả về giá trị mặc định là giây
        try:
            return float(time_str)
        except ValueError:
            raise ValueError(f"Không thể chuyển đổi thời gian: {time_str}")

# Áp dụng hàm convert_to_seconds vào cột 'Response Time'
df['Response Time (seconds)'] = df['Response Time'].apply(convert_time_to_seconds)

# # Kiểm tra lại dữ liệu
df.head()

ValueError: could not convert string to float: '481.089µ'

In [18]:
import pandas as pd
import re

# Đọc dữ liệu từ file CSV, đảm bảo tách cột đúng cách (tùy thuộc vào dấu phân cách trong file của bạn)
df = pd.read_csv('requests1.csv', sep=',')  # Sử dụng dấu phẩy làm phân cách (hoặc sử dụng sep='\t' nếu tab)

# Đảm bảo Timestamp được chuyển sang kiểu datetime đúng
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%Y/%m/%d - %H:%M:%S')

# Chuyển đổi thời gian về giây
def convert_time_to_seconds(time_str):
    # Loại bỏ các khoảng trắng và đảm bảo chuỗi có dạng chuẩn
    time_str = time_str.strip()

    # Kiểm tra và sửa các trường hợp có ký tự không hợp lệ
    time_str = re.sub(r'µ', 'µs', time_str)  # Thay 'µ' thành 'µs'

    # Kiểm tra các đơn vị thời gian và chuyển đổi về giây
    if 's' in time_str:
        # Nếu có đơn vị giây (s), tách số trước 's'
        if 'm' in time_str:
            # Trường hợp có cả phút (m) và giây (s) như "2m6s"
            minutes = int(time_str.split('m')[0])  # lấy phút
            seconds = float(time_str.split('m')[1].replace('s', ''))  # lấy giây
            return minutes * 60 + seconds
        else:
            # Trường hợp chỉ có giây
            return float(time_str.replace('s', ''))
    elif 'ms' in time_str:
        # Trường hợp mili giây (ms), chuyển thành giây
        return float(time_str.replace('ms', '')) / 1000
    elif 'µs' in time_str:
        # Trường hợp micro giây (µs), chuyển thành giây
        return float(time_str.replace('µs', '')) / 1_000_000
    else:
        # Nếu không có đơn vị, trả về giá trị mặc định là giây
        try:
            return float(time_str)
        except ValueError:
            raise ValueError(f"Không thể chuyển đổi thời gian: {time_str}")

# Kiểm tra tên các cột trong dữ liệu để xác định cột "Response Time"
print(df.columns)

# Áp dụng hàm convert_to_seconds vào cột 'Response Time'
# Giả sử cột 'Response Time' có tên đúng là 'Response Time'
df['Response Time (seconds)'] = df['Response Time'].apply(convert_time_to_seconds)

# Kiểm tra lại dữ liệu
print(df.head())


Index(['Timestamp', 'Status', 'Response Time', 'Client IP', 'Method',
       'Endpoint'],
      dtype='object')


ValueError: could not convert string to float: '481.089µ'

In [3]:
print("Tổng số lượng requests tới server: \n", df.count())

Tổng số lượng requests tới server: 
 Timestamp        11874
Status           11874
Response Time    11874
Client IP        11874
Method           11874
Endpoint         11874
dtype: int64


In [12]:
print("Số lượng các Method: ", df["Method"].nunique())
print(df["Method"].unique())

Số lượng các Method:  3
['GET' 'POST' 'HEAD']


In [13]:
print("Số lượng các API Endpoint: ", df["Endpoint"].nunique())
print(df["Endpoint"].unique())

Số lượng các API Endpoint:  9
['/api/tags' '/api/version' '/api/chat' '/api/show' '/api/generate'
 '/api/embed' '/' '/favicon.ico' '/api/ps']


In [ ]:
# # Tính toán các chỉ số thống kê cho Response Time
# response_stats = df['Response Time (ms)'].describe()

# # Hiển thị các chỉ số thống kê
# print("Các chỉ số thống kê về Response Time:")
# print(response_stats)


In [ ]:
# # Tìm các lệnh có thời gian phản hồi chậm nhất
# slowest_requests = df.loc[df['Response Time (ms)'] == df['Response Time (ms)'].max()]

# # Hiển thị các lệnh có phản hồi chậm nhất
# print("Các lệnh có thời gian phản hồi chậm nhất:")
# print(slowest_requests)


In [ ]:
# # Thêm cột 'Second' để nhóm theo mỗi giây
# df['Second'] = df['Timestamp'].dt.floor('S')

# # Đếm số lượng request mỗi giây
# tps = df.groupby('Second').size()

# # Vẽ biểu đồ TPS
# tps.plot(kind='line', figsize=(10, 6))
# plt.title('TPS (Transaction Per Second)')
# plt.xlabel('Thời gian')
# plt.ylabel('Tần suất gửi API')
# plt.show()


In [ ]:
# Thống kê số lượng tin nhắn của mỗi thành viên
count = df['Client IP'].nunique()
message_count = df['Client IP'].value_counts(ascending=True)

# Xuất kết quả
print("Số lượng IP:", count)
print("Thống kê số lần gửi request:")
print(message_count)

# # Lưu kết quả vào file Excel (tùy chọn)
# message_count.to_excel('message_statistics.xlsx')

In [ ]:
# Thống kê tần suất theo Method và Endpoint
method_stats = df.groupby(['Method', 'Endpoint']).size().unstack().fillna(0)

# Hiển thị kết quả thống kê
print("Thống kê tần suất gửi API theo Method và Endpoint:")
print(method_stats)
